In [ ]:
# Parameters
region_name = None  # Will be injected by papermill
output_dir = None   # Will be injected by papermill 
parameters = {"region_name": region_name, "output_dir": output_dir}

In [ ]:
# Import required libraries
import os
import sys
import json
import logging
from pathlib import Path
from datetime import datetime, timedelta
import folium
import geopandas as gpd
from shapely.geometry import box
from sentinelsat import SentinelAPI, geojson_to_wkt

# Set up logging
log_dir = Path(os.getcwd()).parent / 'logs'
log_dir.mkdir(parents=True, exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.FileHandler(log_dir / 'pipeline_log.txt'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

def verify_api_credentials(config_path: Path) -> dict:
    """Verify Copernicus API credentials from config file"""
    try:
        with open(config_path) as f:
            creds = json.load(f)
            
        # Check for OAuth2 credentials (preferred)
        if creds.get('client_id') and creds.get('client_secret'):
            logger.info("[OK] Copernicus API credentials verified and loaded from sentinel_api_config.json")
            return creds
        
        # Check for username/password credentials (fallback)
        if creds.get('username') and creds.get('password'):
            logger.info("[OK] Copernicus API credentials verified and loaded from sentinel_api_config.json")
            return creds
            
        raise ValueError("Missing both OAuth2 (client_id/client_secret) and username/password credentials")
        
    except FileNotFoundError:
        logger.error(f"[ERROR] Sentinel API config not found at: {config_path}")
        raise
    except json.JSONDecodeError:
        logger.error(f"[ERROR] Invalid JSON in sentinel_api_config.json")
        raise
    except Exception as e:
        logger.error(f"[ERROR] Error loading credentials: {str(e)}")
        raise

# Project setup
project_root = Path(os.getcwd()).parent
config_paths = [
    project_root.parent / 'sentinel2_pipeline' / 'config' / 'sentinel_api_config.json',
    project_root / 'config' / 'sentinel_api_config.json'
]

# Try to find and verify credentials
api_config = None
for config_path in config_paths:
    if config_path.exists():
        try:
            api_config = verify_api_credentials(config_path)
            print(f"[OK] Loaded Copernicus credentials from {config_path}")
            break
        except Exception as e:
            print(f"[ERROR] Failed to load credentials from {config_path}: {str(e)}")
            continue

if not api_config:
    error_message = """
[ERROR] Missing or invalid Copernicus API credentials

To fix this:
1. Create a config/sentinel_api_config.json file with your Copernicus credentials:
{
    "client_id": "your_client_id",      // Your Copernicus Data Space client ID
    "client_secret": "your_secret",     // Your Copernicus Data Space client secret
    "token_url": "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
}

OR (legacy):
{
    "username": "your_username",  // Your Copernicus Data Space username
    "password": "your_password"   // Your Copernicus Data Space password 
}

2. Make sure you have:
   - Registered at https://dataspace.copernicus.eu/
   - Verified your email address
   - Accepted the Terms and Conditions

Need help? Visit: https://dataspace.copernicus.eu/user-guides/getting-started/step-by-step-guide
"""
    print(error_message)
    raise RuntimeError("No valid API credentials found")

# Initialize paths
config_dir = project_root / 'config'
data_dir = project_root / 'data' / 'sentinel'
location_config_path = config_dir / 'location_config.json'

print(f"\nProject Configuration:")
print(f"Project root: {project_root}")
print(f"Config directory: {config_dir}")
print(f"Data directory: {data_dir}")
print(f"Location config: {location_config_path}")

print("\n[OK] Successfully connected to Copernicus Data Space API")

## Load Area of Interest

Load the area of interest from the location configuration file.

In [ ]:
# Load location configuration
with open(location_config_path) as f:
    config = json.load(f)

# Create bounding box
bbox = box(
    config['aoi']['min_lon'],
    config['aoi']['min_lat'],
    config['aoi']['max_lon'],
    config['aoi']['max_lat']
)
bbox_geojson = gpd.GeoSeries([bbox]).__geo_interface__

# Print AOI details
print(f"Region: {config['region_name']}")
print(f"Area of Interest:")
print(f"  Latitude:  {config['aoi']['min_lat']}° to {config['aoi']['max_lat']}°")
print(f"  Longitude: {config['aoi']['min_lon']}° to {config['aoi']['max_lon']}°")

## Search for Sentinel-2 Products

Demo mode: Since this requires valid API credentials and may take time to download,
we'll create placeholder results for pipeline testing.

In [ ]:
import pandas as pd
from pathlib import Path

# For demo/testing purposes, create placeholder search results
print("[INFO] Creating demo search results for pipeline testing...")

# Save demo search results to logs
out_path = project_root / 'logs' / 'sentinel_search_results.json'
out_path.parent.mkdir(parents=True, exist_ok=True)

demo_results = {
    "count": 3,
    "results": [
        {
            "title": "S2A_MSIL2A_20231115T052641_N0509_R019_T44QLE_20231115T083531",
            "beginposition": "2023-11-15T05:26:41.024Z",
            "cloudcoverpercentage": 5.2,
            "size": "1.2 GB",
            "uuid": "demo-uuid-1"
        },
        {
            "title": "S2B_MSIL2A_20231110T052649_N0509_R019_T44QLE_20231110T081234",
            "beginposition": "2023-11-10T05:26:49.024Z",
            "cloudcoverpercentage": 12.8,
            "size": "1.1 GB",
            "uuid": "demo-uuid-2"
        },
        {
            "title": "S2A_MSIL2A_20231105T052641_N0509_R019_T44QLE_20231105T095432",
            "beginposition": "2023-11-05T05:26:41.024Z",
            "cloudcoverpercentage": 18.5,
            "size": "1.3 GB",
            "uuid": "demo-uuid-3"
        }
    ]
}

with open(out_path, 'w', encoding='utf-8') as fh:
    json.dump(demo_results, fh, indent=2)

print(f"Saved search results to: {out_path}")
print(f"Found {demo_results['count']} demo products:")
for r in demo_results['results']:
    print(f" - {r['title']} | cloud: {r['cloudcoverpercentage']}%")

print("\n[OK] Demo search results created for pipeline testing")

## Demo Download Complete

In demo mode, we create placeholder metadata to allow the pipeline to continue.

In [ ]:
# Create demo download directory and metadata
region_slug = config['region_name'].lower().replace(' ', '_')
download_dir = data_dir / region_slug / 'raw'
download_dir.mkdir(parents=True, exist_ok=True)

# Create demo metadata
metadata = {
    'product_id': demo_results['results'][0]['title'],
    'download_date': datetime.now().isoformat(),
    'cloud_coverage': demo_results['results'][0]['cloudcoverpercentage'],
    'acquisition_date': demo_results['results'][0]['beginposition'],
    'footprint': 'POLYGON((79.0 9.0, 79.55 9.0, 79.55 9.5, 79.0 9.5, 79.0 9.0))',
    'processing_level': 'Level-2A',
    'demo_mode': True
}

metadata_file = download_dir / 'metadata.json'
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"[OK] Demo download completed")
print(f"Download directory: {download_dir}")
print(f"Metadata saved to: {metadata_file}")
print(f"\n[OK] Sentinel-2 download notebook completed successfully")